In [45]:
import re 
import urllib.request as urllib_request
import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

In [46]:
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

## Rotina de scraping
### Declara-se a variável cards
cards = []

###
url = 'https://fiis.com.br/lista-de-fundos-imobiliarios/'
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0'}
headers = {'User-Agent': 'xxxx'}
req = Request(url, headers = headers)
#req = Request(url)
response = urlopen(req)
html = response.read()
html = html.decode('utf-8')
html = trata_html(html)


In [47]:
## Criação de um objeto Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())
tickets = soup.findAll('span', {'class': 'ticker'})
codigos = []
for ticket in tickets:
    codigos.append(ticket.get_text().lower())

In [48]:
codigo = codigos[0]
url = 'https://fiis.com.br/' + codigo
headers = {'User-Agent': 'xxxx'} # olhar no seu PC
req = Request(url, headers = headers)
response = urlopen(req)
html = response.read()
html = html.decode('utf-8')
html = trata_html(html)
soup = BeautifulSoup(html, 'html.parser')
items = soup.findAll('span', {'class': 'value'})
items = soup.find_all(class_ = 'value')
infos = []
card = {}
for item in items:
    infos.append(item.get_text())

In [49]:
for codigo in codigos[0:10]:
    url = 'https://fiis.com.br/' + codigo
    headers = {'User-Agent': 'xxxx'} # olhar no seu PC
    req = Request(url, headers = headers)
    response = urlopen(req)
    html = response.read()
    html = html.decode('utf-8')
    html = trata_html(html)
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all(class_ = 'value')
    infos = []
    card = {}
    for item in items:
        infos.append(item.get_text())
    card['dividend_yield'] = infos[0]
    card['ultimo_rendimento'] = infos[1]
    card['patrimonio_liquido'] = infos[2]
    card['valor_patrimonial_por_cota'] = infos[3]
    card['telefone'] = infos[4]
    card['email'] = infos[5]
    card['site'] = infos[6]
    card['name'] = infos[7]
    card['tipo_FII'] = infos[8]
    card['tipo_ANBIMA'] = infos[9]
    card['registro_CVM'] = infos[10]
    card['numero_cotas'] = infos[11]
    card['numero_cotistas'] = infos[12]
    card['CNPJ'] = infos[13]
    card['cotacao'] = infos[14]
    card['min_52_weeks'] = infos[15]
    card['max_52_weeks'] = infos[16]
    card['return_12_months'] = infos[17]
    card['ticker'] = codigo.upper()
    
    admin = soup.findAll("div", {"class": "text-wrapper"})
    card['Administrador'] = re.sub("ADMINISTRADOR|[0-9].*[0-9]", "", admin[0].get_text())
    
    # cotacao = soup.findAll("div", {"class": "item quotation"})
    # cotacao = cotacao[0].find("span", {"class": "value"}).get_text()
    
    # min52 = soup.findAll("div", {"class": "item min52"})
    # min52 = min52[0].find("div", {"class": "value"}).get_text()

    # max52 = soup.findAll("div", {"class": "item max52"})
    # max52 = max52[0].find("div", {"class": "value"}).get_text()

    # valorizacao12 = soup.findAll("div", {"class": "item val12"})
    # valorizacao12 = valorizacao12[0].find("div", {"class": "value"}).get_text()
    
    # card["Cotacao Atual"] = cotacao
    # card["Min. 52 semanas"] = min52
    # card["Max. 52 semanas"] = max52
    # card["Valorizacao"] = valorizacao12
    
    cards.append(card)

In [50]:
## Cria-se um dataframe com os resultados
dataset = pd.DataFrame(cards)

In [ ]:
dataset

In [52]:
COLUMNS_LIST = ['ultimo_rendimento',
                'valor_patrimonial_por_cota',
                'numero_cotas',
                'numero_cotistas',
                'cotacao',
                'min_52_weeks',
                'max_52_weeks']
dataset = dataset.apply(lambda x: x.str.replace("R\\$|\\.","", regex = True).str.replace(",", ".").astype(float) if x.name in COLUMNS_LIST else x)
dataset["p_vp"] = dataset['cotacao'] / dataset['valor_patrimonial_por_cota']
#dataset.to_csv('C:/Users/marco/Desktop/Estatistica/fiis_dataset.csv', sep = ';', index = False)


In [ ]:
dataset.head()